<a href="https://colab.research.google.com/github/vvithurshan/Homology-Modelling-for-Abcov-Dataset/blob/main/Structure_for_mutated_sequence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install modeller
!pip install modeller.automodel


In [ ]:
import pandas as pd
from modeller import *
from modeller.automodel import *
# from modeller.automodel import *    

In [ ]:
ls

In [ ]:
df = pd.read_csv("AB-bind_out.csv")

In [ ]:
df.head()

In [ ]:
df = df.drop(columns='Unnamed: 0')

In [ ]:
a = df['Template_Fasta'][0]

In [ ]:
dict = eval(a[1:-1])

In [ ]:
'/'.join(sorted(dict.values(), reverse=True))

In [ ]:
for i in sorted(dict, reverse=False):
  print(i,":", dict[i])

In [ ]:
import os
import shutil
import pandas as pd
import subprocess
from Bio import pairwise2


def ChangeDir(PDB, Unique_id):
  if not os.path.exists(PDB):
    os.makedirs(PDB)

  os.chdir(PDB)

  if not os.path.exists("Homology_Modelling"):
    os.makedirs("Homology_Modelling")
  
  os.chdir("Homology_Modelling")

  if not os.path.exists("Modeller_test"):
    os.makedirs("Modeller_test")

  os.chdir("Modeller_test")

  if not os.path.exists(Unique_id):
    ## raise error
    os.makedirs(Unique_id)
  if not os.path.exists(Unique_id):
    os.makedirs(Unique_id)
  os.chdir(Unique_id)

def SequenceWrite(TemplateFasta, MutatedFasta, PDB):
  ## getting chains in a proper order
  template_name = "{}_template.pdb".format(PDB)
  template_path = "../../../template/" + template_name
    
  u = mda.Universe(template_path)
  chain_ids = [i.segid for i in u.segments]
    
  ###
  temp = eval(TemplateFasta[1:-1])
  TemplateFasta = '/'.join([temp[i] for i in chain_ids])

  mutated = eval(MutatedFasta[1:-1])
  MutatedFasta = '/'.join([mutated[i] for i in chain_ids])
    
  alignments = pairwise2.align.globalxx(TemplateFasta, MutatedFasta)

  TemplateFasta = alignments[0].seqA
  MutatedFasta = alignments[0].seqB
  

  with open("sequence.ali", "w") as fp:
    fp.write(">P1;complex_seq\n")
    fp.write("sequence::.:.:.:.::::\n")
    fp.write(MutatedFasta + "*\n")
    temp = "{}_template.pdb".format(PDB)
    fp.write(f">P1;" +temp + "\n")
    fp.write(f"structure:" + temp + ":FIRST:@:LAST:.::::\n")
    fp.write(TemplateFasta+"*\n")

def Modeller(PDB, Unique_id):
  template_name = "{}_template.pdb".format(PDB)
  template_path = "../../../template/" + template_name
    
  
  if os.path.isfile(template_path):
    pass
  else:
      raise ValueError(
          'Impossible to find template file %s' % (PDB))
  shutil.copy(template_path, os.getcwd())

  ## modeller 
  env = environ()
          
  a = automodel(env, alnfile='sequence.ali',
                knowns= template_name, sequence='complex_seq')

  a.starting_model = 0
  a.ending_model = 0
#   a.auto_align()

#   a.md_level = modeller.automodel.refine.slow
  a.md_level = None
#   a.repeat_optimization = 2
#   a.md_level = refine.slow
#   a.max_var_iterations = 300
  # a.repeat_optimization = 2
#   a.max_molpdf = 1e6
  a.make()

  output = "{}.pdb".format(Unique_id)
  shutil.copy("complex_seq.B99990000.pdb", output)

  filename = output
  name = 'vvithurshan'
  new_name = 'V _VITHURSHAN'

  subprocess.run(['sed', '-i', f's/{name}/{new_name}/g', filename])
  
#   storing_location = '/home/vvithurshan/0014/vvarenthirarajah/Documents/ML-Antibody-Antigen-2023/Homology Modeling/march/P001/AB_Bind_HM_PDB_files'
  storing_location = '/home/vvithurshan/0014/vvarenthirarajah/Documents/ML-Antibody-Antigen-2023/Homology Modeling/march/P001/AB_Bind_HM_PDB_files_test'
  shutil.copy(output, storing_location)

  return 0

def main(row):
  ## Data extraction
  Unique_id = row['Unique_id']
  PDB = row['PDB_Complex']
  TemplateFasta = row['Template_Fasta']
  MutatedFasta = row['Mutated_Fasta']

 ## Go into the Unique_id dir
  ChangeDir(PDB, Unique_id)

  ## Write the sequence
  SequenceWrite(TemplateFasta, MutatedFasta, PDB)

  Modeller(PDB, Unique_id)

if __name__ == "__main__":
  ## load the dataset

  root_dir = "/home/vvithurshan/0014/vvarenthirarajah/Documents/ML-Antibody-Antigen-2023/Homology Modeling/march/"
  root_proj = "/home/vvithurshan/0014/vvarenthirarajah/Documents/ML-Antibody-Antigen-2023/Homology Modeling/march/P001"
    
  os.chdir(root_dir)
  df = pd.read_csv("AB-bind_out.csv")
  
#   if not os.path.exists(root_proj):
#     os.makedirs(root_proj)
  os.chdir(root_proj)

  for index, row in df.iterrows():
    os.chdir(root_proj)
    print(index)
    main(row)
#     if index == 2:
#         break
  os.chdir(root_dir)

In [ ]:
!rm -rf /content/Project_001/

In [ ]:
import MDAnalysis as mda
 
pdb_path = "/home/vvithurshan/0014/vvarenthirarajah/Documents/ML-Antibody-Antigen-2023/Homology Modeling/march/P001/1DQJ/Homology_Modelling/Modeller/1DQJ_23915/1DQJ_template.pdb"
    
u = mda.Universe(pdb_path)
chain_ids = [i.segid for i in u.segments]
chain_ids